# Broadcast

Nous allons voir ici un concept utiliser avec Spark Core et utilisé sous une autre forme avec Spark SQL. Il s'agit des variables _broadcast_.

Nous sommes dans une situation où nous avons récupéré une dataset et nous voulons croiser ses données avec celle d'un référentiel. Par exemple, nous avons récupéré des données de commandes client. Sauf que dans le champ client, nous n'avons que des identifiants et pas l'identité du client. Cette relation entre identité et identifiant du client est fourni dans un autre fichier. On pourrait charger ce fichier dans un RDD, mais il est possible que ce fichier des identifiants soit suffisamment petit pour tenir en mémoire et qu'il peut s'avérer plus intéressant de copier ses données dans chaque exécuteur.

C'est dans ce cas que les variables _broadcast_ peuvent être utilisées.

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("Sales Analysis")
    .getOrCreate()
}

import spark.implicits._
val sparkContext = spark.sparkContext

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

## Chargement d'un fichier de mapping

Dans le code ci-dessous, nous allons charger un fichier contenant des correspondances entre identifiant client et nom de client.

In [ ]:
import scala.util.Using
import scala.io.Source

val mappingFilename = "client-mapping.csv"

val mapping =
    Using(Source.fromFile(mappingFilename)) { file =>
      (
        for (line <- file.getLines().drop(1)) yield {
          val fields = line.split(",")
          fields(0) -> fields(1)
        }
      ).toMap
    }.get

## Chargement des commandes clients

In [ ]:
val rawData: RDD[String] = sparkContext.textFile("orders.csv", 4)

val header: String    = rawData.first()
val data: RDD[String] = rawData.filter(line => line != header)

import java.time._
import java.time.format._

case class Order(
  id:        String,
  clientId:  String,
  timestamp: LocalDateTime,
  product:   String,
  price:     Double
)

def toLocalDateTime(field: String): LocalDateTime =
  LocalDateTime.parse(
    field,
    DateTimeFormatter.ISO_LOCAL_DATE_TIME
  )

def lineToOrder(line: String): Order = {
  val fields = line.split(",")
  Order(
    id = fields(0),
    clientId = fields(1),
    timestamp = toLocalDateTime(fields(2)),
    product = fields(3),
    price = fields(4).toDouble,
  )
}

val orders: RDD[Order] = data.map(lineToOrder)

orders.showHTML(limit=10, truncate=120)

## Broadcast

Une variable _broadcast_ se crée à partir du SparkContext, en utilisant la méthode `.broadcast()` en passant en paramètre la valeur à diffuser sur les exécuteurs.

In [ ]:
val broadcastMapping = ???

La récupération de la valeur associée à une variable _broadcast_ se fait en appelant la méthode `.value`.

Comme notre table de correspondance `broadcastMapping` représente une collection de type `Map` et que nous souhaitons récupérer le nom d'un client par rapport à son identifiant (`order.clientId`), s'il est présent, nous allons utiliser la méthode `.getOrElse(key, default)` pour récupérer le nom du client.

In [ ]:
val mappedOrders =
  orders.map(order =>
    order.copy(clientId = ???)
  )

Nous obtenons maintenant cet affichage :

In [ ]:
mappedOrders.showHTML(limit=10, truncate=120)